In [ ]:
%pip install langchain_community neo4j scikit-learn

In [ ]:
%pip install scikit-learn

In [10]:
import os
#Chaves para modelos
os.environ['OPENAI_API_KEY'] = "YOUR OPENAI KEY"
os.environ["NEO4J_URI"] = "YOUR NEO4J URI"
os.environ["NEO4J_USERNAME"] = "neo4j" 
os.environ["NEO4J_PASSWORD"] = "YOUR NEO4J KEY"

# 3. Criação do grafo

In [ ]:
import tqdm
import os
from langchain_community.graphs import Neo4jGraph
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = 'neo4j'

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

graph = Neo4jGraph()
kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

In [ ]:
kg.refresh_schema()
schema = kg.schema
print(schema)

Node properties:
candidato {nome: STRING, partido: STRING, coalizao: STRING, vice: STRING}
plano {source: STRING, doc_id: STRING}
eixo {nome: STRING, Header_1: STRING, texto: STRING, STD_Header: STRING, Header_2: STRING}
ponto {nome: STRING, texto: STRING, tipo: STRING}
Relationship properties:

The relationships:
(:candidato)-[:planeja]->(:plano)
(:plano)-[:contem]->(:eixo)
(:eixo)-[:propoe]->(:ponto)
(:eixo)-[:seq]->(:eixo)
(:ponto)-[:relacionado_com]->(:ponto)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_most_similar_pairs(list_A, list_B):
    # Compute cosine similarity between all pairs
    similarities = cosine_similarity(list_A, list_B)
    
    # Find the most similar pairs
    most_similar_pairs = []
    for i, row in enumerate(similarities):
        j = np.argmax(row)
        most_similar_pairs.append((i, j, row[j]))
    
    return most_similar_pairs

# Example usage
list_A = [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]]
list_B = [[0.1, 0.2, 0.4], [0.4, -0.5, 0.6], [0.7, 0, 0.9]]

most_similar_pairs = find_most_similar_pairs(list_A, list_B)
for pair in most_similar_pairs:
    print(f"Index in list A: {pair[0]}, Index in list B: {pair[1]}, Cosine Similarity: {pair[2]:.4f}")


Index in list A: 0, Index in list B: 0, Cosine Similarity: 0.9915
Index in list A: 1, Index in list B: 0, Cosine Similarity: 0.9450
Index in list A: 2, Index in list B: 0, Cosine Similarity: 0.9244


In [ ]:
with open('entidades_full.pkl', 'rb') as f:
    final_json = pickle.load(f)

with open('STD_documents.pkl', 'rb') as f:
    all_docs = pickle.load(f)

In [ ]:
for entity in tqdm(final_json, desc="outer loop"):

    # Criar candidato se não ainda houver
    doc = [doc for doc in all_docs if doc.id == entity['doc_id']][0]
    cypher = 'MERGE (n:candidato { nome: "' + doc.metadata['candidato'] + '"}) ON CREATE SET n.nome = "' + doc.metadata['candidato'] + '", n.partido = "' + doc.metadata['partido'] + '", n.coalizao = "' + doc.metadata['coalizao'] + '", n.vice = "' + doc.metadata['vice'] + '"'
    kg.query(cypher)

   # Criar plano de governo, se ...
    cypher = 'MERGE (n:plano { source: "' + doc.metadata['source'] + '"}) ON CREATE SET n.source = "' + doc.metadata['source'] +'", n.doc_id = "' + entity['doc_id'] + '"'
    kg.query(cypher)

    # Ligar Candidato e plano
    cypher = 'MATCH (n1:candidato {nome: "' + doc.metadata['candidato'] + '"}), (n2:plano {source: "' + doc.metadata['source'] + '"}) MERGE (n1)-[hasRelationship:planeja]->(n2)'
    kg.query(cypher)

    # Criar chunks (eixos)
    nome_eixo = entity['doc_id'] + "- " + doc.metadata['STD_Header']
    cypher = 'MERGE (n:eixo { nome: "' + nome_eixo + '"}) ON CREATE SET n.nome = "' + nome_eixo + '", n.Header_1 = "' + doc.metadata['Header_1'] + '", n.texto = "' + doc.page_content + '", n.STD_Header = "' + doc.metadata['STD_Header'] + '"'
    kg.query(cypher)
    if "Header_2" in doc.metadata:
        cypher = 'MATCH (n:eixo) WHERE n.nome = "' + nome_eixo + '" SET n.Header_2 = "' + doc.metadata['Header_2'] + '"'
        kg.query(cypher)

    #Ligar Chunks (eixo) ao plano
    cypher = 'MATCH (n1:plano {source: "' + doc.metadata['source'] + '"}), (n2:eixo {nome: "' + nome_eixo + '"}) MERGE (n1)-[hasRelationship:contem]->(n2)'
    kg.query(cypher)

    # Sequencia chuncks
    if int(entity['doc_id'][-2:]) > 0:
        id1 = entity['doc_id'][0:-2] + f"{(int(doc.id[-2:])-1):02d}"
        cypher = 'MATCH (n1:eixo {nome: "' + id1 + "- " + doc.metadata['STD_Header'] + '"}), (n2:eixo {nome: "' + nome_eixo + '"}) MERGE (n1)-[hasRelationship:seq]->(n2)'
        kg.query(cypher)


    #Criar valores e propostas e suas interrelações
    for element in entity['elements']:
        if element['item_type'] == 'node':
            # Criar node
            cypher = 'MERGE (n:ponto { nome: "' + element['node_name'] + '"}) ON CREATE SET n.nome = "' + element['node_name'] +'", n.tipo = "' + element['node_type'] +'", n.candidato = "' + doc.metadata['candidato'] + '", n.texto = "' + element['node_description'] + '"'
            kg.query(cypher)
            # Ligar node com eixo
            cypher = 'MATCH (n1:eixo {nome: "' + nome_eixo + '"}), (n2:ponto {nome: "' + element['node_name'] + '"}) MERGE (n1)-[hasRelationship:propoe]->(n2)'
            kg.query(cypher)
        else:
            #Ligar sub Chunks
            cypher = 'MATCH (n1 {nome: "' + element['head_node'] + '"}), (n2 {nome: "' + element['tail_node'] + '"}) MERGE (n1)-[hasRelationship:relacionado_com]->(n2)'
            kg.query(cypher)


outer loop: 100%|██████████| 481/481 [31:25<00:00,  3.92s/it]  


In [ ]:
# Criar text index para nomes de propostas e valores e eixos

cypher = "CREATE FULLTEXT INDEX `proposta_tidx` IF NOT EXISTS FOR (n:ponto) ON EACH [n.nome]"
kg.query(cypher)
cypher = "CREATE FULLTEXT INDEX `eixo_tidx` IF NOT EXISTS FOR (n:eixo) ON EACH [n.STD_Header]"
kg.query(cypher)


[]

In [ ]:
# Criar embedings para texto dos pontos (propostas e valores)
cypher = "CREATE VECTOR INDEX ponto_vidx IF NOT EXISTS FOR (n:ponto) ON (n.textEmbedding) OPTIONS { indexConfig: {`vector.dimensions`: 3072, `vector.similarity_function`: 'cosine' }}"
kg.query(cypher)
cypher = 'MATCH (n:ponto) WHERE n.textEmbedding IS NULL WITH n, genai.vector.encode(n.texto, "OpenAI", {token:"' + OPENAI_API_KEY + '", model:"text-embedding-3-large"}) AS vector CALL db.create.setNodeVectorProperty(n, "textEmbedding", vector)'
kg.query(cypher)

In [ ]:
kg.refresh_schema()
schema = kg.schema
print(schema)

Node properties:
candidato {nome: STRING, partido: STRING, coalizao: STRING, vice: STRING}
plano {source: STRING, doc_id: STRING}
eixo {nome: STRING, Header_1: STRING, texto: STRING, STD_Header: STRING, Header_2: STRING}
ponto {nome: STRING, texto: STRING, tipo: STRING}
Relationship properties:

The relationships:
(:candidato)-[:planeja]->(:plano)
(:plano)-[:contem]->(:eixo)
(:eixo)-[:propoe]->(:ponto)
(:eixo)-[:seq]->(:eixo)
(:ponto)-[:relacionado_com]->(:ponto)


In [ ]:
# Calcula a similaridade entre cosenos de cada par de pontos nas propostas
def find_most_similar_pairs(list_A, list_B):
    
    similarities = cosine_similarity(list_A, list_B)
    
    # Find the most similar pairs
    most_similar_pairs = []
    for i, row in enumerate(similarities):
        j = np.argmax(row)
        most_similar_pairs.append((i, j, row[j]))
    
    return most_similar_pairs

# Lista de planos ('doc_ids)
cypher="MATCH (p:plano) RETURN p.doc_id"
planos = kg.query(cypher)

# Para cada plano
# coletar lista de embeddings: [{'plano': 'doc_i', 'embeddings', []}]
embeddings = []
for plano in tqdm(planos):
    doc_id = plano['p.doc_id']
    response = kg.query("MATCH (p:plano {doc_id:'" + doc_id + "'})-[:contem]->(:eixo)-[:propoe]->(pto:ponto) RETURN elementId(pto), pto.textEmbedding")
    id_list = []
    embedding_list = []
    for r in response:
        id_list.append(r['elementId(pto)'])
        embedding_list.append(r['pto.textEmbedding'])
    embeddings.append({'ids':id_list, 'embeddings':embedding_list})

# Criar ligações contendo a distância como propriedade
for i in tqdm(range(len(planos))):
    for j in range(len(planos)):
        if not planos[i]['p.doc_id'] == planos[j]['p.doc_id']:
            nearest_neighbors = find_most_similar_pairs(embeddings[i]['embeddings'], embeddings[j]['embeddings'])
            # Criar relações no grafo
            for n in nearest_neighbors:
                cypher = "MATCH (a:ponto), (b:ponto) WHERE elementId(a)='" + embeddings[i]['ids'][n[0]] + "' AND elementId(b)='" + embeddings[j]['ids'][n[1]] + "' CREATE (a)-[:similar_to {cos_similarity:" + str(n[2]) + "}]->(b)"
                kg.query(cypher)